# Principal Component Anlaysis and Cointegration
This article outlines the relation between Principal Component Anlaysis (PCA) and cointegration based on Chigira (2008). 

Recall that a prerequisite of cointegrated series is that every series has to be integrated of the same order. Let's denote <a href="https://en.wikipedia.org/wiki/Order_of_integration">I(d)</a> for integration of order d. This means after d times of taking difference (i.e. $x_{t} - x_{t-1}$), the series becomes <a href="https://en.wikipedia.org/wiki/Stationary_process#Weak_or_wide-sense_stationarity">(_weak_) stationary</a>. For simplicity, we will explore the case of I(1) series only here. In this case, cointegration simply says that there exists a linear combination of these integrated series such that the combination is stationary.

Further recall that <a href="https://en.wikipedia.org/wiki/Principal_component_analysis">(_linear_) Principal Component Analysis</a> seeks to find a set of vectors that maximize the variance after the vectors are applied as weights to the original series. What immediately follows from this is:
- The maximized variance is the eigenvalue of the eigen-decomposition of the variance-covariance matrix of the original data. 
- The eigenvectors are the weights. These vectors may also be known as _**factor loading**_ in other fields of studies.
- A principal component is the linear combination of the original series with eigenvectors as the weights.
- The 2nd (or 3rd, 4th, ...) largest variance is the eignvalue of the variance-covariance matrix based on the residual data that the previous principal component(s) cannot explain. 
- Because these eigenvectors are <a href="https://en.wikipedia.org/wiki/Orthonormality">orthonormal</a> (i.e. the dot product is 0 betwen any two and 1 with  itself), the principal components are orthogonal as well.

Chigira (2008) proved that if the last few principal components are stationary, the original data are in fact cointegrated, provided each series is I(1). Furthermore, the number of stationary principal components equals the number of cointegration vectors as well as the number of PCA eigenvectors.

Let $\pmb{x}_{t}=[x^1_t, x^2_t, ..., x^l_t ]^\prime$ denote a vector of variables from $x^1$ to $x^l$ at time period $t$, properly demeaned and detrended. Just as Chigira (2008), we use a VAR model with a constant and time trend and no lag to model out the mean and trend:
\begin{equation*}
    \pmb{y}_t = \mu + \sigma t + \pmb{x}_{t}
\end{equation*}
where $\pmb{x}_{t}$ is simply the residual of VAR on the original series $\pmb{y}_t$.

Let $\pmb{\Beta} = [ \pmb{v}_1, ... , \pmb{v}_m]$ be an $m$ by $l$ matrix of PCA eigenvectors, where $m$ is the intended number of principal components to keep. Thus $m \le l$. The columns of $\pmb{\Beta}$ are ordered descendingly according to the eigenvectors' eigenvalues. The PCA components are then defined as:
\begin{align*}
    \pmb{V}_t &= \pmb{\Beta}^\prime \pmb{x}_{t}\\
    &= \begin{bmatrix}
        | & & |\\
        \pmb{v}_1^\prime \pmb{x}_{t}  &... & \pmb{v}_m^\prime \pmb{x}_{t}\\
        | & & |\\
    \end{bmatrix}
\end{align*}

Chigira cointegration test then becomes a test of stationary principal components. This implementation utilizes the following precedure:
1. $i=0$, $r=0$
2. Run a <a href="https://en.wikipedia.org/wiki/Dickey%E2%80%93Fuller_test">Dicky-Fuller</a> unit root test on the principal component with $i$-th _smallest_ eigenvalue<br>
3. If the test shows unit root, return $r$
4. If the test does not show a unit root, let $r = i + 1$
5. Unless $i$ = $m$, increase $i$ by 1

The return variable $r$ represents the number of cointegrated vectors as in <a href="https://en.wikipedia.org/wiki/Johansen_test">Johansen cointegration</a> test.

As Lin et al. (2019) noted, the benifits of the Chigira cointegration test includes:
- The Chigira test can handle a large number of variables even with a small number of time periods
- The Chigira test is more flexible than Johansen cointegrationtest since the latter assumes an autoregressive structure

Below is an example code to test cointegration of a series of data.

Let $\pmb X$ denote an $n$ by $l$ matrix of data. Each column represents a time series. Each row represents a time period. 

\begin{equation}
    \pmb{\Beta} = 
    \begin{bmatrix}
            \pmb{v}_1\\
            \pmb{v}_2\\
    \end{bmatrix}
\end{equation}

- The <a href="https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html">sklearn's PCA</a> implmentation produces orthonormal eigenvectors, meaning they are of unit length on top of being orthogonal. 

In [19]:
import numpy as np

a = np.array([
    [-0.007, 0.017, 0.386, 0.423, -.081]
    , [-.01, .023,.544,.595,-.114]
    , [-.368, -.641, -.478, .47, .072]
    , [0, -.001, -.018, -.019, .004]
    , [-.489, -.483, .532, -.49, -.074]
    , [-.559, .426, .039, .074, .707]
    , [-.556, .417, -.204, .029, -.686]
])
# a = np.array([[-.77,.27], [.557, -.161], [0,0], [.313,.949]])
a

array([[-0.007,  0.017,  0.386,  0.423, -0.081],
       [-0.01 ,  0.023,  0.544,  0.595, -0.114],
       [-0.368, -0.641, -0.478,  0.47 ,  0.072],
       [ 0.   , -0.001, -0.018, -0.019,  0.004],
       [-0.489, -0.483,  0.532, -0.49 , -0.074],
       [-0.559,  0.426,  0.039,  0.074,  0.707],
       [-0.556,  0.417, -0.204,  0.029, -0.686]])

In [20]:
a.T.dot(a)

array([[ 9.963110e-01,  1.740000e-03, -7.630000e-04,  2.490000e-04,
        -2.400000e-03],
       [ 1.740000e-03,  1.000354e+00,  8.000000e-05, -8.800000e-05,
         7.070000e-04],
       [-7.630000e-04,  8.000000e-05,  9.999010e-01, -1.070000e-03,
         3.790000e-04],
       [ 2.490000e-04, -8.800000e-05, -1.070000e-03,  1.000632e+00,
         3.550000e-04],
       [-2.400000e-03,  7.070000e-04,  3.790000e-04,  3.550000e-04,
         1.000678e+00]])

\begin{equation}
    
\end{equation}

## Reference
- Chigira, H. (2008). A test of cointegration rank based on principal component analysis. Applied Economics Letters, 15(9), 693-696.
- Lin, Y., Kruger, U., Gu, F., Ball, A., & Chen, Q. (2019). Monitoring nonstationary and dynamic trends for practical process fault diagnosis. Control Engineering Practice, 84, 139-158.